In [5]:
from neal import SimulatedAnnealingSampler
import numpy as np
import dimod
from collections import defaultdict
import matplotlib.pyplot as plt
import networkx as nx
from time import time
import timeit 
from docplex.mp.model import Model
from openqaoa.problems import FromDocplex2IsingModel

sampler_sa = SimulatedAnnealingSampler()



In [151]:
def ising_from_weights_terms(problem):
    h = {}
    J = {}
    for term, weight in problem.items():
        if len(term) == 1:
            h[term[0]] = weight
        else:
            J[term[0],term[1]] = weight
    return h, J

def energy(x, hamiltonian):
    obj = 0
    spin = {"1":-1, "0":1}
    for k, v in hamiltonian.items():
        if len(k) == 2:
            obj += v * spin[x[k[0]]] * spin[x[k[1]]]
        elif len(k) == 1:
            obj += v * spin[x[k[0]]]
        else:
            print(k, v)
    return obj

def objective(samples_dict, hamiltonian, sol):
    optimal_energy = energy(sol, hamiltonian)
    results = []
    probability = 0
    for bitstring, counts in samples_dict.items():
        energy_sample = energy(bitstring, hamiltonian)
        r  = energy_sample/optimal_energy
        if np.abs(energy_sample - optimal_energy) < 1e-8:
            probability += counts
        if optimal_energy - energy_sample > 1e-10:
            print(f"There is a better cost than that of CPLEX by {optimal_energy - energy_sample}!")
        results.append([energy_sample, r, counts])
    results = np.array(results)
    shots = np.sum(results[:,2])
    mean_r = np.sum(results[:,0] * results[:,2])/(shots*optimal_energy)
    probability /= shots
    return {"results":np.array(results), "optimal_cost":optimal_energy, "r":mean_r, "probability":probability}

def objective_no_sol(samples_dict, hamiltonian):
    results = []
    probability = 0
    min_energy = 0
    for bitstring, counts in samples_dict.items():
        energy_sample = round(energy(bitstring, hamiltonian),6)
        if energy_sample < min_energy:
            min_energy = energy_sample
            probability = counts
            sol = bitstring
        elif energy_sample == min_energy:
            probability += counts
        results.append([energy_sample, counts])
    results = np.array(results)
    shots = np.sum(results[:,1])
    probability /= shots
    return {"results":np.array(results), "min_cost":min_energy, "probability":probability, "sol":sol}


def MaxCut_mdl(G):
    """
    Input:
        G: Networkx graph
    Output:
        mdl: Docplex model
    """
    # MIS model as a QUBO problem
    mdl = Model('MaxCut')
    num_vertices = G.number_of_nodes()
    x = {i: mdl.binary_var(name=f"x_{i}") for i in range(num_vertices)}
    cost = 0
    for i, j in G.edges:
        if nx.is_weighted(G):
            cost += - G[i][j]["weight"] * (x[i] * (1 - x[j]) + x[j] * (1 - x[i]))
        else:
            cost += - (x[i] * (1 - x[j]) + x[j] * (1 - x[i]))
    mdl.minimize(cost)
    return mdl

def MIS_mdl(G, penalty=2):
    # MIS model as a QUBO problem
    mdl = Model('MIS')
    num_vertices = G.number_of_nodes()

    x = {i: mdl.binary_var(name=f"x_{i}") for i in range(num_vertices)}
    mdl.minimize(-mdl.sum(x) + penalty * mdl.sum(
        x[i] * x[j] for (i, j) in G.edges
    ))
    return mdl

def running_sa(hamiltonian, shots, sol, num_sweeps):
    h , J =  ising_from_weights_terms(hamiltonian) 
    bqm = dimod.BQM.from_ising(h, J)
    ti = time()
    r = sampler_sa.sample(bqm, num_sweeps=num_sweeps, num_reads=shots)
    tf = time() - ti
    df = r.to_pandas_dataframe().sort_values('energy').reset_index(drop=True)
    samples = defaultdict(int)
    for _, row in df.iterrows():
        s_d = ''.join('0' if row[q] == 1 else '1' for q in row.keys() if type(q) == int)
        samples[s_d] += row["num_occurrences"]
    if sol:
        results = objective(samples, hamiltonian, sol)
    else:
        results = objective_no_sol(samples, hamiltonian)
    pd = 0.99
    results["STS"] = max(np.ceil(np.log(1 - pd) / np.log(1 - results["probability"])),1) if results["probability"] < 1 else 1
    results["Nevals"] = num_sweeps * results["STS"]
    results["time"] = tf
    return results

def generate_random_WMC(nq):
    G = nx.Graph()
    G.add_nodes_from(range(nq))

models = {"MIS":MIS_mdl, "WMaxCut":MaxCut_mdl, "3MaxCut":MaxCut_mdl}



In [10]:
def compute_energy(spins, hamiltonian):
    """
    Compute the energy of the given spin configuration based on the Hamiltonian.
    """
    energy = 0.0
    for edges, coupling in hamiltonian.items():
            if len(edges) == 1:
                 energy += coupling * spins[edges[0]]
            elif len(edges) == 2:
                energy += coupling * spins[edges[0]] * spins[edges[1]]
    return energy # To account for double counting

def simulated_annealing(hamiltonian, nq, initial_temp=10, final_temp=0.1, sweeps=100):
    """
    Perform simulated annealing to find a low-energy configuration.
    """
    spins = {key: np.random.choice([-1, 1]) for key in range(nq)}  # Random initial spin configuration
    previous_energy = compute_energy(spins, hamiltonian)
    temperatures = np.linspace(initial_temp, final_temp, sweeps)
    nevals = 1
    for temperature in temperatures:
        for spin_to_flip in range(nq):
            spins[spin_to_flip] *= -1  # Flip the spin
            new_energy = compute_energy(spins, hamiltonian)
            delta_energy = new_energy - previous_energy
            
            if delta_energy > 0 and np.random.rand() > np.exp(-delta_energy / temperature):
                spins[spin_to_flip] *= -1  # Revert flip if move is not accepted
            else:
                previous_energy = new_energy   
            nevals += 1 
    return spins, nevals

def customized_running_sa(hamiltonian, shots, sol, num_sweeps, initial_temp=0.5, final_temp=0.01):
    nq = len(sol)
    samples = defaultdict(int)
    ti = timeit.timeit()
    for _ in range(shots):
        final_spins, nevals = simulated_annealing(hamiltonian, nq, initial_temp=initial_temp, final_temp=final_temp, sweeps=num_sweeps)
        s = {-1:"0", 1:"1"}
        samples["".join(s[final_spins[i]] for i in range(nq))] += 1
    tf = timeit.timeit() - ti
    results = objective(samples, hamiltonian, sol)
    pd = 0.99
    results["Nevals"] = nevals
    results["probability"] = results["probability"]
    results["STS"] = max(np.ceil(np.log(1 - pd) / np.log(1 - results["probability"])),1) if results["probability"] < 1 else 1
    results["time"] = tf
    return results

In [ ]:
problems = np.load(f"./Data/WMaxCut_cte_no_mdl.npy", allow_pickle=True).item()
shots = 1000
results = np.load(f"./Data/WMC_SA_results.npy", allow_pickle=True).item()
arg_sort_dict = np.load("./Data/hard_FC_WMC.npy", allow_pickle=True).item()

problem_name = "WMaxCut"
# results = {}
nqs = [20,25,30,35,40]
ps = [50,100,200,300,400,500]
for nodes in nqs: 
    print(f"number of nodes {nodes}")
    results[nodes] = {}
    arg_sort = arg_sort_dict[nodes]
    for kk in problems[nodes].keys():#arg_sort:
        if not kk % 10:
            print(f"----- k: {kk} ------")
        results[nodes][kk] = defaultdict(dict)
        G = problems[nodes][kk]["G"]
        H = problems[nodes][kk]["ising_hamiltonian"]
        hamiltonian = {tuple(k):v for k,v in zip(H.terms, H.weights) if np.abs(v) > 1e-6}
        mdl = models[problem_name](G)
        ti = time()
        mdl.solve() # solve the problem using CLPEX
        tf = time() - ti
        results[nodes][kk]["n_iterations_cplex"] = mdl.get_solve_details().nb_iterations
        results[nodes][kk]["time_cplex"] = mdl.get_solve_details().deterministic_time
        results[nodes][kk]["real_time_cplex"] = mdl.get_solve_details().time
        results[nodes][kk]["time_cplex_func"] = tf
        results[nodes][kk]["sol"] = "".join(str(round(mdl.solution.get_value(var))) for var in mdl.iter_binary_vars())
        results[nodes][kk]["sol"] = problems[nodes][kk]["sol"]
        results[nodes][kk]["SA"] = {}
        for p in ps:
            results[nodes][kk]["SA"][p] = running_sa(hamiltonian, shots, results[nodes][kk]["sol"], p)
        

number of nodes 20
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 25
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 30
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 35
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 ------
----- k: 80 ------
----- k: 90 ------
number of nodes 40
----- k: 0 ------
----- k: 10 ------
----- k: 20 ------
----- k: 30 ------
----- k: 40 ------
----- k: 50 ------
----- k: 60 ------
----- k: 70 -----

In [166]:
np.save(f"./Data/WMC_SA_results.npy", results)